# EXTRACCION DE PRODUCTOS Y PRECIOS POR CATEGORIA

El objetivod este notebook es optener una lista de productos y pecios con el fin de darle realizmo a la generación del dataset sintetico de gastos personales.


In [ ]:
%pip install beautifulsoup4 pandas selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import re
from datetime import datetime
import time
import os

In [ ]:
# Diccionario de categorías y URLs
categorias = {
    "carnes": "https://catalogo.candymarket.com.pe/search?lineid=11",
    "embutidos": "https://catalogo.candymarket.com.pe/search?lineid=12",
    "carnes_aderezadas_y_especerias": "https://catalogo.candymarket.com.pe/search?lineid=13",
    "lacteos_y_huevos": "https://catalogo.candymarket.com.pe/search?lineid=14",
    "frutas_y_verduras": "https://catalogo.candymarket.com.pe/search?lineid=15",
    "panaderia_y_pasteleria": "https://catalogo.candymarket.com.pe/search?lineid=16",
    "patio_de_comidas": "https://catalogo.candymarket.com.pe/search?lineid=18",
    "abarrotes": "https://catalogo.candymarket.com.pe/search?lineid=21",
    "licores_y_gaseosas": "https://catalogo.candymarket.com.pe/search?lineid=22",
    "golosinas": "https://catalogo.candymarket.com.pe/search?lineid=23",
    "jugos_y_nectares": "https://catalogo.candymarket.com.pe/search?lineid=24",
    "limpieza": "https://catalogo.candymarket.com.pe/search?lineid=25",
    "tocador_y_aseo_personal": "https://catalogo.candymarket.com.pe/search?lineid=26",
    "menaje": "https://catalogo.candymarket.com.pe/search?lineid=32",
    "ferreteria": "https://catalogo.candymarket.com.pe/search?lineid=33",
    "farmacia": "https://catalogo.candymarket.com.pe/search?lineid=43"
}

In [5]:
# Crear carpeta para los archivos CSV
CARPETA_PRODUCTOS = "productos"
if not os.path.exists(CARPETA_PRODUCTOS):
    os.makedirs(CARPETA_PRODUCTOS)
    print(f"📁 Carpeta '{CARPETA_PRODUCTOS}' creada\n")

📁 Carpeta 'productos' creada



In [6]:
def limpiar_precio(precio_text):
    """Extrae el precio numérico del texto"""
    if not precio_text:
        return ""
    # Busca patrón S/ seguido de números
    match = re.search(r'S/\s*(\d+\.?\d*)', precio_text)
    if match:
        return match.group(1)
    return ""

In [ ]:
def extraer_unidad(texto):
    """Extrae la unidad de medida del texto buscando el patrón 'x UNIDAD'"""
    if not texto:
        return ""
    
    # Buscar patrón "x UNIDAD" (ej: "x UN", "x KG", "x PA", "x TR")
    # Puede aparecer como "x UN" o "X UN"
    match = re.search(r'[xX]\s+([A-Z]{2,3})\b', texto)
    if match:
        unidad_encontrada = match.group(1).upper()
        # Mapear las unidades a formato estándar
        mapeo_unidades = {
            'UN': 'und',
            'UND': 'und',
            'KG': 'kg',
            'GR': 'gr',
            'LT': 'lt',
            'ML': 'ml',
            'PA': 'paquete',
            'TR': 'tira',
            'FR': 'frasco',
            'BT': 'botella',
            'LA': 'lata',
            'PQ': 'paquete'
        }
        return mapeo_unidades.get(unidad_encontrada, unidad_encontrada.lower())
    
    # Fallback: buscar directamente las palabras clave
    texto_upper = texto.upper()
    if 'X KG' in texto_upper:
        return 'kg'
    elif 'X UN' in texto_upper:
        return 'und'
    elif 'X GR' in texto_upper:
        return 'gr'
    elif 'X PA' in texto_upper:
        return 'paquete'
    elif 'X TR' in texto_upper:
        return 'tira'
    elif 'X LT' in texto_upper:
        return 'lt'
    elif 'X ML' in texto_upper:
        return 'ml'
    
    return ""

In [8]:
def configurar_driver():
    """Configura el driver de Chrome"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
    
    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [9]:
def extraer_productos_categoria(driver, url_categoria):
    """Extrae productos de una categoría específica usando Selenium"""
    productos = []
    fecha = datetime.now().strftime('%Y-%m-%d')
    
    try:
        driver.get(url_categoria)
        
        # Esperar a que los productos se carguen
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "col")))
        
        # Esperar tiempo adicional para asegurar carga completa
        time.sleep(5)
        
        # Buscar todos los contenedores de productos
        cards = driver.find_elements(By.CSS_SELECTOR, "div.col.p-2")
        
        for card in cards:
            try:
                # Obtener todo el texto de la card
                card_text = card.text.strip()
                
                # Método 1: Buscar nombre por ID
                nombre = ""
                try:
                    nombre_elem = card.find_element(By.CSS_SELECTOR, "span[id*='lblTitulo']")
                    nombre = nombre_elem.text.strip()
                except:
                    pass
                
                # Método 2: Si no encuentra, buscar por clase
                if not nombre:
                    try:
                        nombre_elem = card.find_element(By.CSS_SELECTOR, "p.single-line")
                        nombre = nombre_elem.text.strip()
                    except:
                        pass
                
                # Método 3: Extraer del texto completo
                if not nombre and card_text:
                    lineas = card_text.split('\n')
                    for linea in lineas:
                        if linea and len(linea) > 5 and 'NUEVO' not in linea and 'SUPER' not in linea and 'SOLO' not in linea and 'Precio' not in linea and 'S/' not in linea and 'x ' not in linea.lower():
                            nombre = linea.strip()
                            break
                
                if not nombre or len(nombre) < 3:
                    continue
                
                # Extraer precio del texto completo
                precio = ""
                precio_match = re.search(r'S/\s*(\d+\.?\d*)', card_text)
                if precio_match:
                    precio = precio_match.group(1)
                
                # Extraer unidad del texto usando la función mejorada
                unidad = extraer_unidad(card_text)
                
                # Solo agregar si tiene al menos nombre y precio
                if nombre and precio:
                    productos.append({
                        "fecha_extraccion": fecha,
                        "nombre_producto": nombre,
                        "unidad_medida": unidad,
                        "precio_por_unidad": precio,
                    })
            
            except Exception as e:
                continue
        
        return productos
    
    except Exception as e:
        print(f"❌ Error en {url_categoria}: {e}")
        return []

In [10]:
# MAIN: Procesar todas las categorías
print("="*70)
print("EXTRACCIÓN DE PRODUCTOS - CANDY MARKET")
print("="*70)
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📁 Carpeta de salida: {CARPETA_PRODUCTOS}/")
print("="*70 + "\n")

driver = None
total_productos = 0
categorias_exitosas = 0

try:
    driver = configurar_driver()
    
    for idx, (categoria, url) in enumerate(categorias.items(), 1):
        print(f"[{idx}/{len(categorias)}] 📦 {categoria.upper()}", end=" ... ")
        
        productos = extraer_productos_categoria(driver, url)
        
        if productos:
            df = pd.DataFrame(productos)
            filename = os.path.join(CARPETA_PRODUCTOS, f"{categoria}.csv")
            df.to_csv(filename, index=False, encoding="utf-8-sig")
            
            print(f"✅ {len(productos)} productos")
            total_productos += len(productos)
            categorias_exitosas += 1
        else:
            print(f"⚠️  0 productos")
        
        time.sleep(2)

finally:
    if driver:
        driver.quit()

print("\n" + "="*70)
print("RESUMEN DE EXTRACCIÓN")
print("="*70)
print(f"✅ Categorías procesadas: {categorias_exitosas}/{len(categorias)}")
print(f"📊 Total de productos extraídos: {total_productos}")
print(f"📁 Archivos guardados en: {CARPETA_PRODUCTOS}/")
print("="*70)

EXTRACCIÓN DE PRODUCTOS - CANDY MARKET
📅 Fecha: 2025-12-02 11:20:57
📁 Carpeta de salida: productos/

[1/16] 📦 CARNES ... ✅ 55 productos
[2/16] 📦 EMBUTIDOS ... ✅ 43 productos
[3/16] 📦 CARNES_ADEREZADAS_Y_ESPECERIAS ... ✅ 29 productos
[4/16] 📦 LACTEOS_Y_HUEVOS ... ✅ 53 productos
[5/16] 📦 FRUTAS_Y_VERDURAS ... ✅ 57 productos
[6/16] 📦 PANADERIA_Y_PASTELERIA ... ✅ 30 productos
[7/16] 📦 PATIO_DE_COMIDAS ... ✅ 22 productos
[8/16] 📦 ABARROTES ... ✅ 259 productos
[9/16] 📦 LICORES_Y_GASEOSAS ... ✅ 77 productos
[10/16] 📦 GOLOSINAS ... ✅ 58 productos
[11/16] 📦 JUGOS_Y_NECTARES ... ✅ 12 productos
[12/16] 📦 LIMPIEZA ... ✅ 121 productos
[13/16] 📦 TOCADOR_Y_ASEO_PERSONAL ... ✅ 76 productos
[14/16] 📦 MENAJE ... ✅ 11 productos
[15/16] 📦 FERRETERIA ... ✅ 21 productos
[16/16] 📦 FARMACIA ... ✅ 5 productos

RESUMEN DE EXTRACCIÓN
✅ Categorías procesadas: 16/16
📊 Total de productos extraídos: 929
📁 Archivos guardados en: productos/
